- 以下只介绍不同部分
- 环绕音效无非就是增加工作量，需要对音频进行分段，用不同方位的冲激串进行卷积，最后相加，达到环绕的效果

In [ ]:
!pip install numpy scipy matplotlib

In [2]:
import numpy as np
import wave
import scipy.io.wavfile as wav
from scipy.signal import resample, convolve


In [3]:

# 读取wav文件
def read_wav(filename):
    Fs, data = wav.read(filename)
    return Fs, data


In [4]:

# 写入wav文件
def write_wav(filename, Fs, data):
    wav.write(filename, Fs, data.astype(np.int16))


In [5]:

# 重采样函数
def resample_audio(data, original_fs, new_fs):
    num_samples = int(len(data) * float(new_fs) / original_fs)
    return resample(data, num_samples)


In [6]:

# 处理单声道音频进行卷积
def process_audio(L,R, hrir_l, hrir_r, Fs, new_fs):
    # 重采样音频
    resampled_audio_L = resample_audio(L,Fs,new_fs)
    resampled_audio_R = resample_audio(R,Fs,new_fs)
    # 分别与左右HRIR卷积
    out_l = convolve(resampled_audio_L, hrir_l)
    out_r = convolve(resampled_audio_R, hrir_r)
    
    # 合成立体声
    out_stereo = np.column_stack((out_l, out_r))
    return out_stereo


- 读入多个方位的冲激串

In [ ]:
change=[[0,'000'],[10,'040'],[20,'080'],[30,'120'],[40,'161'],[50,'200'],[60,'240'],[70,'285'],[80,'300'],[90,'000']]

L_path=[f'full\\elev{i[0]}\\L{i[0]}e{i[1]}a.wav' for i in change]
R_path=[f'full\\elev{i[0]}\\R{i[0]}e{i[1]}a.wav' for i in change]
# 读取HRIR数据
HL=[]
HR=[]
for i in range(10):
    Fs_hrir, hrir_l = read_wav(L_path[i])
    _, hrir_r = read_wav(R_path[i])
    hrir_l,hrir_r=hrir_l*0.00005,hrir_r*0.00005
    HL.append(hrir_l)
    HR.append(hrir_r)
print(max(hrir_l),max(hrir_r))

In [ ]:

# 读取音频文件
Fs, audio_data = read_wav('test1.wav')
print(Fs)
if len(audio_data[0])==2:
    audio_data_LL=[data[0] for data in audio_data]
    audio_data_RR=[data[1] for data in audio_data]
    print("双声道")
else:
    print('单声道')
print(max(audio_data_LL),max(audio_data_RR))

- 分段进行音频处理，最后相加

In [9]:
# 设置目标采样率
new_fs = 48000
out=None
num=len(audio_data_LL)//10
# 处理音频，进行卷积并生成立体声
for i in range(10):
    newl=audio_data_LL[i*num:(i+1)*num]
    newr=audio_data_RR[i*num:(i+1)*num]
    hiri_l=HL[i]
    hiri_r=HR[i]
    processed_audio = process_audio(newl,newr,hiri_l, hiri_r, Fs, new_fs)
    if i!=0:
        out=np.append(out,processed_audio,axis=0)
    else:
        out = processed_audio
# 将处理后的音频写入文件
write_wav('processed_audio.wav', new_fs, out)
